In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib as mpl
import numpy as np
import glob
import astropy.units as u
import tables
import gc
import time

from astropy.coordinates import SkyCoord, AltAz
from astropy.coordinates.erfa_astrom import ErfaAstromInterpolator, erfa_astrom
from astropy.time import Time
from lstchain.reco.utils import add_delta_t_key, get_effective_time
from os import path
from scipy.stats import binned_statistic
from pyirf.statistics import li_ma_significance
from lstchain.spectra.crab import crab_magic
from ctapipe.containers import EventType
from pyirf.spectral import CRAB_MAGIC_JHEAP2015
from ctapipe.coordinates import CameraFrame, TelescopeFrame
from gammapy.stats import WStatCountsStatistic
from ctapipe_io_lst import LSTEventSource
from ctapipe.io import read_table

%matplotlib inline

In [ ]:
#
# This is a notebook to calculate the flux sensitivity using real Crab observations.
#
# NOTE: the inputs of this notebook are DL2 files of Crab observations, 
# both for source-independent and source-dependent analyses.
# 
# The source-independent files must contain the event-wise nominal position 
# of Crab (src_x, src_y), in a table called "source_position". This can be 
# achieved by processing standard DL2 files with $LSTCHAIN/scripts/lstchain_dl2_add_sourcepos.py
# (this is done like this because it is too costly to compute the position every time we need it)
#
#
# The samples below are those used for the ApJ LST-1 performance (or "Crab") paper:
# NOTE!! The full sample (34.2 h of effective time) takes about 3 hours to process in the IT cluster!
# The time is mostly to read the data and fill the "super-histograms" from which the event statistics
# are derived. One can fill the histograms once, and the run only the (much faster) second part of the 
# notebook with different settings for the sensitivity calculation.
#
# source-independent dataset
tag1 = "source-independent"
#dataset1 = glob.glob("/fefs/aswg/workspace/abelardo.moralejo/Crab_performance_paper/data_v0.9.9/DL2/process_with*/dl2_LST-1.Run*.h5")
dataset1 = glob.glob("/fefs/aswg/workspace/abelardo.moralejo/Crab_performance_paper/data_v0.9.9/DL2_sourcepos_with_v010/dl2_LST-1.Run*.h5")

# Source-dependent dataset
# Created with RFs trained using gammas out to 1 deg, AND additional src-indep params:
tag2 = "source-dependent+, ring-wobble-0-1-trained"
dataset2 = glob.glob("/fefs/aswg/workspace/seiya.nozaki/Crab_performance_paper/20221027_v0.9.9_crab_tuned/combined_off_axis_1deg/DL2/data/dl2_LST-1.Run*.h5")

dataset1.sort()
dataset2.sort()

In [ ]:
#
# Settings for sensitivity calculations:
# (NOTE: one can change these afterwards, to obtain results with different conditions just
# running the second part of the notebook, with no need to re-read all data!!!)
#
alpha = 0.2 # Li & Ma's "alpha", ratio of the ON to the OFF exposure
# This what we *assume* for the calculation, because it is the standard value.
# Note however that for a single LST, and to avoid large systematics, we use only one off
# region to estimate the background (in source-dependent analysis; in source-independent 
# analysis it is one region up to 250 GeV and 3 regions for >250 GeV). We then "extrapolate"
# that background to what we would have in case we really had alpha=0.2...


# Assumed (effective) observation time for the calculation (note: it has nothing to do with 
# the actual observation time of the sample used for the calculation!):
obs_time = 50 * u.h 
# obs_time = 1 * u.min
# obs_time = 5 * u.min
# obs_time = 0.5 * u.h
# obs_time = 0.7 * u.h
# obs_time = 5 * u.h

# The 5-sigma condition is so standard that we do not make it configurable! It is hardcoded. 
# Additional sensitivity conditions:
min_n_gammas = 10 # Minimum number of excess events 
min_backg_percent = 5 # Excess must be at least this percentage of the background


In [ ]:
# Binnins of gammaness, Alpha, thetas and energy for all calculations:

# Very fine binnings to allow cut optimization:
gammaness_bins = np.linspace(0, 1, 2001)
alphabins = np.linspace(0, 60, 3001)
theta2bins = np.linspace(0, 0.6, 6001)

logenergy_bins = np.linspace(-2.0, 2.2, 22) # 5 per decade

evt_id_bins = [-0.5, 0.5, 1.5]   
# for event_id%2 (to separate odd & even event numbers). 
# Even event-id's end up in the first bin; odd event-id's in the second one.
# We separate them so that we can optimize the cuts in one and apply them to 
# the other (and vice versa)

# Background estimation:
# For source-independent analysis, we will use one off region up to 250 GeV. 
# Three off regions for higher energies:
first_ebin_with_3offs = 7 # i.e. from ~250 GeV onwards
# NOTE!  This is just what is used to *estimate* the average background in the on-region. 
# For the actual computation of sensitivity, in order to follow the standard definition, 
# we use (Li & Ma's) alpha = 0.2  (define in the cell above), i.e. we assume we can compute 
# the background in a region 5 times larger than the on-region. In practice it is somehow 
# optimistic for a standalone telescope like LST-1, with its limited angular resolution.
#

# Intensity cuts:
min_intensity = 50
max_intensity = 1e10

In [ ]:
#
# CUTS TO ENSURE THE QUALITY OF THE REAL DATA EXCESS FROM WHICH THE SENSITIVITY WILL BE COMPUTED!
#
# We set here the conditions to ensure that all used histogram bins (defined by Ebin, gammaness cut and 
# angular cut) have, in the data sample, a reliable excess which we can use for estimating sensitivity:
#
min_signi = 3   # below this value (significance of the test source, Crab, for the *actual* observation 
                # time of the sample and obtained with 1 off region) we ignore the corresponding cut 
                # combination

min_exc = 0.002 # in fraction of off. Below this we ignore the corresponding cut combination. 
min_off_events = 10 # minimum number of off events in the actual observation used. Below this we 
                    # ignore the corresponding cut combination.

# Note that min_exc is set to a pretty low value! In the observations published in the LST1 performance 
# paper the background at low E is stable to 0.5% (i.e. 0.005)

In [ ]:
#
# For the calculation of the uncertainty of sensitivity:
backg_syst = 0.01 # relative background systematic uncertainty

backg_normalization = False # generally normalization is not needed, background is uniform enough.
norm_range = np.array([[0.1, 0.16], [20., 59.8]]) # deg2 for theta2,  deg for Alpha

In [ ]:
sa = LSTEventSource.create_subarray(tel_id=1)
focal = sa.tel[1].optics.effective_focal_length

# The source position src_x, src_y (in m), stored in "source_position", is calculated by 
# lstchain_dl2_add_sourcepos.py using the effective focal length (29.30565 m), which means 
# that it is "consistent" with the reco values reco_src_x, reco_src_y (which are affected 
# by the telescope's optical aberration)
#
print(focal)

In [ ]:
on_events = [None, None]
off_events = [None, None]

# In on_events and off_events (which are basically 5-axis histograms):
# axis 0 has two bins. Indicates analysis type. index=0: source-independent;  index=1: source-dependent 
#
# axis 1 has two bins. index=0: even event-id events;   index=1: odd-event-id events
# axis 2 is the energy axis, see logenergy_bins above
# axis 3 is the gammaness axis, see gammaness_bins above
# axis 4 is the angular axis (theta or Alpha), see theta2bins and alphabins above


In [ ]:
tablename = "/dl2/event/telescope/parameters/LST_LSTCam"
livetimes = []

livetime = 0


for ifile, file in enumerate(dataset1):

    print(ifile+1, '/', len(dataset1), ': ', file, time.asctime(time.gmtime()))
    
    tb = read_table(file, tablename)

    # See above the note on the source_position table!
    tb_extra = read_table(file, "source_position/table")
    
    lt, _ = get_effective_time(tb)
    livetime += lt

    
    dx = np.rad2deg((tb['reco_src_x']-tb_extra['src_x'])/focal.to_value(u.m))
    dy = np.rad2deg((tb['reco_src_y']-tb_extra['src_y'])/focal.to_value(u.m))
    tb['theta2_on']  = (dx**2 + dy**2).astype('float32')
        
    dx = np.rad2deg((tb['reco_src_x']+tb_extra['src_x'])/focal.to_value(u.m))
    dy = np.rad2deg((tb['reco_src_y']+tb_extra['src_y'])/focal.to_value(u.m))
    tb['theta2_off'] = (dx**2 + dy**2).astype('float32')

    dx = np.rad2deg((tb['reco_src_x']+tb_extra['src_y'])/focal.to_value(u.m))
    dy = np.rad2deg((tb['reco_src_y']-tb_extra['src_x'])/focal.to_value(u.m))
    tb['theta2_off_90'] = (dx**2 + dy**2).astype('float32')
    
    dx = np.rad2deg((tb['reco_src_x']-tb_extra['src_y'])/focal.to_value(u.m))
    dy = np.rad2deg((tb['reco_src_y']+tb_extra['src_x'])/focal.to_value(u.m))
    tb['theta2_off_270'] = (dx**2 + dy**2).astype('float32')
    
    tb['odd_or_even'] = (tb['event_id']%2).astype('float32')


    # Filter to select cosmics (=shower events)
    noped = (tb['event_type'] != EventType.SKY_PEDESTAL.value)
    nocal = (tb['event_type'] != EventType.FLATFIELD.value)
    cosmics = noped & nocal
    
    mask = ((tb['intensity']>min_intensity) & 
            (tb['intensity']<max_intensity) & 
            cosmics)
    
    on, _  = np.histogramdd(np.array([tb['odd_or_even'][mask], 
                                      tb['log_reco_energy'][mask].astype('float32'), 
                                      tb['gammaness'][mask].astype('float32'), 
                                      tb['theta2_on'][mask]]).T,
                            bins=[evt_id_bins, logenergy_bins, gammaness_bins, theta2bins])

    on = on.astype('float32')
    
    
    if on_events[0] is None:
        on_events[0] = on
    else:
        on_events[0] += on


    off, _ = np.histogramdd(np.array([tb['odd_or_even'][mask],
                                      tb['log_reco_energy'][mask].astype('float32'),
                                      tb['gammaness'][mask].astype('float32'), 
                                      tb['theta2_off'][mask]]).T, 
                            bins=[evt_id_bins, logenergy_bins, gammaness_bins, theta2bins])        
    off = off.astype('float32')
        
    # For bins >= first_ebin_with_3offs we fill the off with the average of 3 off regions
    off[:,first_ebin_with_3offs:,:,:] *= 1/3.
    high_e_mask = mask & (tb['log_reco_energy'] >= logenergy_bins[first_ebin_with_3offs])

    off += np.histogramdd(np.array([tb['odd_or_even'][high_e_mask],
                                    tb['log_reco_energy'][high_e_mask].astype('float32'),
                                    tb['gammaness'][high_e_mask].astype('float32'), 
                                    tb['theta2_off_90'][high_e_mask]]).T, 
                          bins=[evt_id_bins, logenergy_bins, gammaness_bins, 
                                theta2bins])[0].astype('float32') / 3.

    off += np.histogramdd(np.array([tb['odd_or_even'][high_e_mask],
                                    tb['log_reco_energy'][high_e_mask].astype('float32'),
                                    tb['gammaness'][high_e_mask].astype('float32'), 
                                    tb['theta2_off_270'][high_e_mask]]).T, 
                          bins=[evt_id_bins, logenergy_bins, gammaness_bins, 
                                theta2bins])[0].astype('float32') / 3.

    if off_events[0] is None:
        off_events[0] = off
    else:
        off_events[0] += off
    
    on = None
    off = None
    tb = None
    tb_extra = None
    gc.collect() # memory clean-up
        

print("Live time:", livetime.to(u.h))
livetimes.append(livetime)


In [ ]:
# Columns to be kept in the case of source-dependent analysis (again, so save memory):

columns_srcdep = ["('on', 'expected_src_x')",
                  "('on', 'expected_src_y')",
                  "('on', 'alpha')",
                  "('off_180', 'alpha')",
                  "('on', 'gammaness')",
                  "('off_180', 'gammaness')",
                  "('on', 'reco_energy')",
                  "('off_180', 'reco_energy')"]
columns = ['obs_id', 
           'event_id',
           'intensity', 
           'event_type',
           'x', 'y', 'psi']


tablename = "/dl2/event/telescope/parameters/LST_LSTCam"
tablename_srcdep = "/dl2/event/telescope/parameters_src_dependent/LST_LSTCam"

livetime = 0

on_events[1] = None
off_events[1] = None

for ifile, file in enumerate(dataset2):

    print(ifile+1, '/', len(dataset2), ':  ', file, time.asctime(time.gmtime()))

    tb = read_table(file, tablename)
    tb['odd_or_even'] = (tb['event_id']%2).astype('float32')

    
    lt, _ = get_effective_time(tb)
    livetime += lt

    tb_srcdep = read_table(file, tablename_srcdep)

    tb_srcdep.rename_columns(["('on', 'expected_src_x')", "('on', 'expected_src_y')",
                              "('on', 'alpha')", "('off_180', 'alpha')",
                              "('on', 'gammaness')", "('off_180', 'gammaness')",
                              "('on', 'reco_energy')", "('off_180', 'reco_energy')"],
                             ['src_x', 'src_y', 'alpha_on', 'alpha_off', 
                              'gammaness_on', 'gammaness_off', 'reco_energy_on', 'reco_energy_off'])

    noped = (tb['event_type'] != EventType.SKY_PEDESTAL.value)
    nocal = (tb['event_type'] != EventType.FLATFIELD.value)
    cosmics = noped & nocal
    
    mask = ((tb['intensity']>min_intensity) & 
            (tb['intensity']<max_intensity) & 
            cosmics)
    
    on, _  = np.histogramdd(np.array([tb['odd_or_even'][mask],
                                      np.log10(tb_srcdep['reco_energy_on'][mask]),
                                      tb_srcdep['gammaness_on'][mask], 
                                      tb_srcdep['alpha_on'][mask]]).T, 
                            bins=[evt_id_bins, logenergy_bins, gammaness_bins, alphabins])
    on = on.astype('float32')

    off, _  = np.histogramdd(np.array([tb['odd_or_even'][mask],
                                      np.log10(tb_srcdep['reco_energy_off'][mask]),
                                      tb_srcdep['gammaness_off'][mask], 
                                      tb_srcdep['alpha_off'][mask]]).T, 
                            bins=[evt_id_bins, logenergy_bins, gammaness_bins, alphabins])
    off = off.astype('float32')

    if on_events[1] is None:
        on_events[1] = on
        off_events[1] = off
    else:
        on_events[1] += on
        off_events[1] += off
    
    on = None
    off = None
    tb = None
    tb_srcdep = None
    gc.collect() # memory cleanup
    

print("Live time:", livetime.to(u.h))
livetimes.append(livetime)


In [ ]:
# Index 0 refers to the source-independent analysis, index 1 to the source-dependent analysis.
# If the Crab data samples are the same (just different analysis method), the live times should 
# be the same:

print(livetimes[0].to(u.h))
print(livetimes[1].to(u.h))

In [ ]:
# Show the excess:
# plt.errorbar(0.5*(bins[1:]+bins[:-1]), onevts-offevts, yerr=(onevts+offevts)**0.5, fmt='o', markersize=3)
# plt.xlabel('Alpha (deg)')
# plt.ylabel('Excess events')
# plt.grid()
# plt.show()

In [ ]:
# Prepare arrays to contain the cumulative sums, from each gammaness value to 1, and from 0 to each 
# value of theta (or Alpha):

cum_on_events = [np.copy(on_events[0]), np.copy(on_events[1])]
cum_off_events = [np.copy(off_events[0]), np.copy(off_events[1])]
excess_events = [on_events[0]-off_events[0], on_events[1]-off_events[1]]

In [ ]:
cum_on_events[0].shape
# 0 indicates source-dependent analysis. 
# The four axis are odd/even event_id's; energy; gammaness_cut; theta2_cut; 

In [ ]:
# Obtain the cumulative histograms: integrate in gammaness and in theta (or alpha).

for evtid in range(cum_on_events[0].shape[0]):
    for energyid in range(cum_on_events[0].shape[1]):
        # gammaness_bins and theta / alpha bins arrays are of bin edges... 
        # Note that actual number of bins in histograms is that number -1

        for i in reversed(range(len(gammaness_bins)-2)):
            cum_on_events[0][evtid, energyid, i,:] += cum_on_events[0][evtid, energyid, i+1,:]
            cum_off_events[0][evtid, energyid, i,:] += cum_off_events[0][evtid, energyid, i+1,:]
            cum_on_events[1][evtid, energyid, i,:] += cum_on_events[1][evtid, energyid, i+1,:]
            cum_off_events[1][evtid, energyid, i,:] += cum_off_events[1][evtid, energyid, i+1,:]

    
        for j in range(len(theta2bins)-1):
            if j == 0:
                continue
            cum_on_events[0][evtid, energyid, :, j] += cum_on_events[0][evtid, energyid, :, j-1]
            cum_off_events[0][evtid, energyid, :, j] += cum_off_events[0][evtid, energyid, :, j-1]

        for j in range(len(alphabins)-1):
            if j == 0:
                continue
            cum_on_events[1][evtid, energyid, :, j] += cum_on_events[1][evtid, energyid, :, j-1]
            cum_off_events[1][evtid, energyid, :, j] += cum_off_events[1][evtid, energyid, :, j-1]

In [ ]:
cum_excess_events = [[],[]]
cum_excess_events[0] = cum_on_events[0] - cum_off_events[0]
cum_excess_events[1] = cum_on_events[1] - cum_off_events[1]

In [ ]:
#
# Just a plot to illustrate the content of the cumulative histograms, and to check the overall good 
# agreement of the on- and off- alpha distributions. Note that it will not be perfect, some gammas 
# may be present in the OFF at middle-Alpha values, especially for soft gammaness cuts!
# source-dependent analysis (1), even event_id's (0)
#
Ebin = 6 # Energy bin
gammanessbin = 1900 # gammaness cut (in bins)
alpha_rebin = 50  # rebinning for better view
ON = np.diff(cum_excess_events[1][0][Ebin][gammanessbin][::alpha_rebin] +
             cum_off_events[1][0][Ebin][gammanessbin][::alpha_rebin])

xx = np.linspace(alphabins.min(), alphabins.max(), len(ON)+1)

plt.plot(xx[:-1], ON, 'o', label='ON')
plt.plot(xx[:-1], np.diff(cum_off_events[1][0][Ebin][gammanessbin][::alpha_rebin]), label='OFF')
plt.ylim(0, 1.1*np.max(ON))
plt.xlabel('Alpha')
plt.ylabel('Events')
plt.grid()
plt.legend()
plt.show()

In [ ]:
# Just a check plot: even event_id's, source-independent analysis, and some specific gammaness & theta2 cuts
plt.plot(cum_on_events[0][0,:,1400,200], label='ON')
plt.plot(cum_off_events[0][0,:,1400,200], label='OFF')
plt.plot(cum_excess_events[0][0,:,1400,200], label='Excess')
plt.yscale('log')
plt.xlabel('Energy bin')
plt.ylabel('Events')
plt.legend()
plt.show()

In [ ]:
# Having excess and OFF we do not need ON:
cum_on_events[0] = None
cum_on_events[1] = None
cum_on_events = None
gc.collect() # memory clean-up

In [ ]:
# Check for size of histograms in GB:
for i, type_str in enumerate(['Source-independent:', 'Source-dependent:']):
    print(type_str, 
          excess_events[i].size*excess_events[i].itemsize/1e9,
          cum_excess_events[i].size*cum_excess_events[i].itemsize/1e9,
          cum_off_events[i].size*cum_off_events[i].itemsize/1e9,
          off_events[i].size*off_events[i].itemsize/1e9,
          on_events[i].size*on_events[i].itemsize/1e9)

In [ ]:
# Look for other large objects in memory (TEST)
# import sys
# sorted([(x, sys.getsizeof(globals().get(x))) for x in dir()], key=lambda x: x[1], reverse=True)

In [ ]:
# Just a test plot
evtid = 1 # Plot odd events 
energyid = 7 # energy bin 7

# Excess events, per bin:
plt.figure(figsize=(16,4))
plt.pcolormesh(theta2bins, gammaness_bins, excess_events[0][evtid][energyid], norm=colors.LogNorm())
plt.colorbar()
plt.xlabel('theta2 (deg2)')
plt.ylabel('gammaness')
plt.show()

# Cumulative off events:
plt.figure(figsize=(16,4))
plt.pcolormesh(theta2bins, gammaness_bins, cum_off_events[0][evtid][energyid], norm=colors.LogNorm())
plt.colorbar()
plt.xlabel('theta2 cut (deg2)')
plt.ylabel('gammaness cut')
plt.show()


# Cumulative excess events:
plt.figure(figsize=(16,4))
plt.pcolormesh(theta2bins, gammaness_bins, cum_excess_events[0][evtid][energyid], norm=colors.LogNorm())
plt.colorbar()
plt.xlabel('theta2 cut (deg2)')
plt.ylabel('gammaness cut')
plt.show()


In [ ]:
#################################################################################################
#                                                                                               #
# SECOND PART OF THE NOTEBOOK!!! RUN FROM HERE IF YOU JUST WANT TO RE-RUN THE CUT OPTIMIZATION  #
# USING THE SAME DATA SAMPLES!                                                                  #
#                                                                                               #
##################################################################################################

In [ ]:
#################################################################################################
#                                                                                               #
# UNCOMMENT BELOW THE SETTINGS YOU WANT TO MODIFY (RELATIVE TO WHAT WAS SET AT THE BEGINNING)   #
#                                                                                               #
#################################################################################################

# alpha = 0.2 # Li & Ma's "alpha", ratio of the ON to the OFF exposure
# This what we *assume* for the calculation, because it is the standard value.
# Note however that for a single LST, and to avoid large systematics, we use only one off
# region to estimate the background (in source-dependent analysis; in source-independent 
# analysis it is one region up to 250 GeV and 3 regions for >250 GeV). We then "extrapolate"
# that background to what we would have in case we really had alpha=0.2...


# Assumed (effective) observation time for the calculation (note: it has nothing to do with 
# the actual observation time of the sample used for the calculation!):
# obs_time = 50 * u.h 
# obs_time = 1 * u.min
# obs_time = 5 * u.min
# obs_time = 0.5 * u.h
# obs_time = 0.7 * u.h
# obs_time = 5 * u.h


# The 5-sigma condition is so standard that we do not make it configurable! It is hardcoded. 
# Additional sensitivity conditions:
# min_n_gammas = 10 # Minimum number of excess events 
# min_backg_percent = 5 # Excess must be at least this percentage of the background
#

In [ ]:
#################################################################################################
#                                                                                               #
# UNCOMMENT BELOW THE SETTINGS YOU WANT TO MODIFY (RELATIVE TO WHAT WAS SET AT THE BEGINNING)   #
#                                                                                               #
#################################################################################################

#
# CUTS TO ENSURE THE QUALITY OF THE REAL DATA EXCESS FROM WHICH THE SENSITIVITY WILL BE COMPUTED!
#
# We set here the conditions to ensure that all used histogram bins (defined by Ebin, gammaness cut and 
# angular cut) have, in the data sample, a reliable excess which we can use for estimating sensitivity:
#
# min_signi = 3   # below this value (significance of the test source, Crab, for the *actual* observation 
                # time of the sample and obtained with 1 off region) we ignore the corresponding cut 
                # combination

# min_exc = 0.002 # in fraction of off. Below this we ignore the corresponding cut combination. 
# min_off_events = 10 # minimum number of off events in the actual observation used. Below this we 
                    # ignore the corresponding cut combination.

# Note that min_exc is set to a pretty low value! In the observations published in the LST1 performance 
# paper the background at low E is stable to 0.5% (i.e. 0.005)

In [ ]:
#
# For the calculation of the uncertainty of sensitivity:
backg_syst = 0.01 # relative background systematic uncertainty

backg_normalization = False # generally normalization is not needed, background is uniform enough.
norm_range = np.array([[0.1, 0.16], [20., 59.8]]) # deg2 for theta2,  deg for Alpha

In [ ]:
#
# Function to compute the fraction of the Crab flux that results in a given significance, for the 
# observation time and Li&Ma's alpha set above.
# This is based on the observed excess and background (cumul_excess and cumul_off). This is done for all 
# bins of cumul_excess and cumul_off, that is, for all possible gammaness and theta2/Alpha cuts
#
# In order to get more reliable results:
# We can require the *actually observed* excess to be a minimum fraction of the background (min_exc)
# We can require the *actually observed* excess to have a significance of at least min_signi (computed
# assuming just 1 off region)
# By "actually observed" we mean that it is not the excess (or significance) extrapolated for a 
# 50h-observation or whatever, but the actually obtained result in the input data sample.
#
def calc_flux_for_N_sigma(N_sigma, cumul_excess, cumul_off, 
                          min_signi, min_exc, min_off_events, alpha,
                          target_obs_time, actual_obs_time):
    
    time_factor = target_obs_time.to_value(u.h) / actual_obs_time.to_value(u.h)

    start_flux = 1
    flux_factor = start_flux * np.ones_like(cumul_excess)

    good_bin_mask = ((cumul_excess > min_exc*cumul_off) &
                     (cumul_off > min_off_events))

    flux_factor = np.where(good_bin_mask, flux_factor, np.nan)
    
    # First calculate significance (with 1 off) of the excesses in the provided sample, with no scaling.
    # We will only use the cut combinations where we have at least min_signi sigmas to begin with...
    # NOTE!!! float64 precision is essential for the arguments of li_ma_significance!

    lima_signi = li_ma_significance((flux_factor*cumul_excess + cumul_off).astype('float64'), 
                                    cumul_off.astype('float64'), 
                                    alpha=1)
            
    # Set nan in bins where we do not reach min_signi:
    flux_factor = np.where(lima_signi > min_signi, flux_factor, np.nan)

    
    # Now calculate the significance for the target observation time_
    lima_signi = li_ma_significance((time_factor*(flux_factor*cumul_excess +
                                                  cumul_off)).astype('float64'), 
                                    (time_factor*cumul_off/alpha).astype('float64'), 
                                    alpha=alpha)

    
    # iterate to obtain the flux which gives exactly N_sigma:
    for iter in range(4):
        # print(iter)
        tolerance_mask = np.abs(lima_signi-N_sigma)>0.001 # recalculate only what is needed
        flux_factor[tolerance_mask] *= (N_sigma / lima_signi[tolerance_mask])
        # NOTE!!! float64 precision is essential here!!!!
        lima_signi[tolerance_mask] = li_ma_significance((time_factor*(flux_factor[tolerance_mask]*
                                                                      cumul_excess[tolerance_mask]+
                                                                      cumul_off[tolerance_mask])).astype('float64'), 
                                                         (time_factor*cumul_off[tolerance_mask]/alpha).astype('float64'), 
                                                         alpha=alpha)
    return flux_factor, lima_signi

In [ ]:
lima_signi = [[], []]
flux_for_5_sigma = [[], []]

# Compute the flux (in Crab fraction), for all cuts, which results in 5 sigma for the sensitivity conditions
# defined above:

for k in range(2):
    flux_for_5_sigma[k], lima_signi[k] = calc_flux_for_N_sigma(5, cum_excess_events[k], cum_off_events[k], 
                                                               min_signi, min_exc, min_off_events, alpha,
                                                               obs_time, livetimes[k]*0.5)
# NOTE: livetime is divided by 2 because calculation is done separately for the odd- and even-event_id samples!

In [ ]:
# Now make sure we only consider bins with valid flux in *both* samples (odd- and even-events).
# This is because we will use the optimal cuts obtained with odd- events to apply them to even- events,
# and vice-versa. If the flux is nan for one of the two, we set it to nan for both.

for k in range(2):
    mask = np.isnan(flux_for_5_sigma[k][0]) | np.isnan(flux_for_5_sigma[k][1])
    flux_for_5_sigma[k] = np.where(mask, np.nan, flux_for_5_sigma[k])
    lima_signi[k] = np.where(mask, np.nan, lima_signi[k])

In [ ]:
# Check that significances are indeed 5 (or very close - they come from an interative procedure)
# Each entry in the histograms is a cut combination.

plt.hist(lima_signi[0].flatten(), bins=500, range=(0, 10), log=True)
plt.xlabel('Li & Ma significance')
plt.show()

plt.hist(lima_signi[1].flatten(), bins=500, range=(0, 10), log=True)
plt.xlabel('Li & Ma significance')
plt.show()

In [ ]:
#
# Function to calculate the flux needed to obtain a given excess, for all cut combinations:
#
def calc_flux_for_N_excess(N_excess, cumul_excess, target_obs_time, actual_obs_time):
    time_factor = target_obs_time.to_value(u.h) / actual_obs_time.to_value(u.h)
    return (N_excess/(cumul_excess*time_factor))

#
# Do the computation (JUST FOR TEST!!):
#
# flux_for_10_excess = [[], []]
# for k in range(2):
#     flux_for_10_excess[k] = calc_flux_for_N_excess(min_n_gammas, cum_excess_events[k], obs_time, livetimes[k]*0.5)
#     # livetime divided by 2 because calculation is done separately for the odd- and even-event_id samples!
    
# flux_for_10_excess = [np.where(np.isnan(flux_for_5_sigma[0]), np.nan, flux_for_10_excess[0]), 
#                       np.where(np.isnan(flux_for_5_sigma[1]), np.nan, flux_for_10_excess[1])]

In [ ]:
#
# Function to calculate the flux needed to obtain a given percent of the background, 
# for all cut combinations:
#
def calc_flux_for_x_percent_backg(percent, cumul_excess, cumul_off):
    # In fraction of the flux of the test source (Crab):
    return percent/100*cumul_off/cumul_excess

#
# Do the computation (JUST FOR TEST!!):
#
# flux_for_5percent_backg = [[], []]
# for k in range(2):
#     flux_for_5percent_backg[k] = calc_flux_for_x_percent_backg(min_backg_percent, cum_excess_events[k], 
#                                                                cum_off_events[k])

# flux_for_5percent_backg = [np.where(np.isnan(flux_for_5_sigma[0]), np.nan, flux_for_5percent_backg[0]), 
#                            np.where(np.isnan(flux_for_5_sigma[1]), np.nan, flux_for_5percent_backg[1])]


In [ ]:
#
# Function to calculate the flux (for all cut combinations) which fulfills
# all 3 conditions of sensitivity (standard: at least 5-sigma significance, 
# at least 10 excess events, and the excess being at least 5% of the background):
#
def calc_flux_3conditions(cumul_excess, cumul_off, 
                          min_signi, min_exc, min_off_events, alpha,
                          target_obs_time, actual_obs_time,
                          min_n_gammas, min_backg_percent):
    
    f1, _ = calc_flux_for_N_sigma(5, cumul_excess, cumul_off, 
                                  min_signi, min_exc, min_off_events, alpha,
                                  target_obs_time, actual_obs_time)
    f2 = 0
    f3 = 0
    
    if min_n_gammas > 0:
        f2 = calc_flux_for_N_excess(min_n_gammas, cumul_excess, target_obs_time, actual_obs_time)
    if min_backg_percent > 0:
        f3 = calc_flux_for_x_percent_backg(min_backg_percent, cumul_excess, cumul_off)
    
    return np.maximum(np.maximum(f1, f2), f3)  

In [ ]:
#
# Definitions of min_signi, min_exc and min_off_events (to consider a cut combination valid) are above
#
# Do the computation of the detectable flux (for all cut combinations) according to the sensitivity conditions:
#
detectable_flux = [[],[]]

#Minimum flux fulfilling all three conditions:
for k in range(2):
    detectable_flux[k] = calc_flux_3conditions(cum_excess_events[k], cum_off_events[k],
                                               min_signi, min_exc, min_off_events, alpha,
                                               obs_time, livetimes[k]*0.5,
                                               min_n_gammas, min_backg_percent)

In [ ]:
# Just in case, make sure we only consider bins with valid flux in *both* samples (odd- and even-events):
for k in range(2):
    mask = np.isnan(detectable_flux[k][0]) | np.isnan(detectable_flux[k][1])
    detectable_flux[k] = np.where(mask, np.nan, detectable_flux[k])

In [ ]:
sensitivity = [[[], []], [[], []]] # [analysis_type, odd_or_even, energy]
reco_energy = [[[], []], [[], []]]
signi = [[[], []], [[], []]]

cut_indices = [[[], []], [[], []]] # [analysis_type, odd_or_even, energy]

# Tweak: sometimes the minimization at low Ereco ends up with very tight alpha cut 
# just because of a fluke... We try to avoid it here, by setting minimum "reasonable" 
# cut values for the low-E bins:
#
min_angle_cut = [np.zeros(len(logenergy_bins)-1), np.zeros(len(logenergy_bins)-1)]
min_angle_cut[0][:5] = 0.02
min_angle_cut[1][:5] = 5

# Now, in each subsample (odd/even event_id's) we will find which cuts provide the best sensitivity
# (= minimum detectable flux), and will apply thoise cuts to the *other* subsample. 

for analysis_type in range(2): # source-independent and source-dependent analyses
    for even_or_odd in range(2):
        # We optimize the cuts with the sample indicated by even_or_odd,
        # and apply them on the complementary sample, "other_half":
        if even_or_odd == 0:
            other_half = 1
        else:
            other_half = 0

        for iebin in range(len(logenergy_bins)-1):
            reco_energy[analysis_type][other_half].append(10**(0.5*(logenergy_bins[iebin]+logenergy_bins[iebin+1])))

            # Now find the cuts which provide the minimum detectable flux using the events with odd event_id.

            # Except if we have only nan values... :
            if np.sum(~np.isnan(detectable_flux[analysis_type][even_or_odd][iebin])) == 0:
                sensitivity[analysis_type][other_half].append(np.nan)
                signi[analysis_type][other_half].append(np.nan)
                cut_indices[analysis_type][other_half].append([0, 0])
                continue

            if analysis_type == 0:
                start_bin = np.where(theta2bins>min_angle_cut[0][iebin])[0][0] - 1
            else:
                start_bin = np.where(alphabins>min_angle_cut[1][iebin])[0][0] - 1
            
            index = np.nanargmin(detectable_flux[analysis_type][even_or_odd][iebin,:,start_bin:])            

            indices = list(np.unravel_index(index, 
                                            detectable_flux[analysis_type][even_or_odd][iebin, :, start_bin:].shape))
            indices[1] += start_bin
            
            # Now get & store the minimum detectable flux with these cuts but using the other half of the events
            # Keep also the best-cut indices for later use

            sensitivity[analysis_type][other_half].append(detectable_flux[analysis_type][other_half][iebin, indices[0], indices[1]])
            signi[analysis_type][other_half].append(lima_signi[analysis_type][other_half][iebin, indices[0], indices[1]])
            cut_indices[analysis_type][other_half].append(indices)

        sensitivity[analysis_type][other_half] = np.array(sensitivity[analysis_type][other_half])
        reco_energy[analysis_type][other_half] = np.array(reco_energy[analysis_type][other_half])
        signi[analysis_type][other_half] = np.array(signi[analysis_type][other_half])
        cut_indices[analysis_type][other_half] = np.array(cut_indices[analysis_type][other_half])
        

In [ ]:
#
# MAGIC sensitivity in Crab fraction:
#
def plot_MAGIC_sensitivity_fraction():
    s = np.loadtxt('magic_sensitivity.txt', skiprows = 1)
    energy = (s[:,0] * u.GeV).to(u.TeV)
    percentage = s[:,5]
    plt.plot(energy, percentage/100.,  '-.', label='MAGIC (stereo) [Aleksić et al. 2016]', color='tab:green')
    return

In [ ]:
plt.figure(figsize=(10,4))
plt.scatter(reco_energy[0][1], sensitivity[0][1], marker='o', facecolors='tab:blue', edgecolors='tab:blue',
            label=tag1+", odd id")
plt.scatter(reco_energy[0][0], sensitivity[0][0], marker='o', facecolors='none', edgecolors='tab:blue',
            label=tag1+", even id")

plt.scatter(reco_energy[1][1], sensitivity[1][1], marker='o', facecolors='tab:orange', edgecolors='tab:orange',
            label=tag2+", odd id")
plt.scatter(reco_energy[1][0], sensitivity[1][0], marker='o', facecolors='none', edgecolors='tab:orange',
            label=tag2+", even id")

plt.yscale('log')
plt.xscale('log')

plt.ylim(0.005, 10)
plt.xlim(0.01, 200)
plt.ylabel("Fraction of Crab flux")
plt.xlabel("Reconstructed energy / TeV")

plot_MAGIC_sensitivity_fraction()

plt.legend()
plt.grid()
plt.show()  

In [ ]:
plt.figure(figsize=(10,4))
plt.scatter(reco_energy[0][1], 0.5*(sensitivity[0][1]+sensitivity[0][0]), 
            marker='o', facecolors='tab:blue', edgecolors='tab:blue',
            label=tag1)

plt.scatter(reco_energy[1][1], 0.5*(sensitivity[1][1]+sensitivity[1][0]), marker='o', facecolors='tab:orange', edgecolors='tab:orange',
            label=tag2)

plt.yscale('log')
plt.xscale('log')

plt.ylim(0.005, 5)
plt.xlim(0.01, 200)
plt.ylabel("Fraction of Crab flux")
plt.xlabel("Reconstructed energy / TeV")

plot_MAGIC_sensitivity_fraction()

plt.legend(loc='lower right')
plt.grid()
plt.show()  

In [ ]:
# Check the significance for the optimal cuts, for all the energy bins:
plt.figure(figsize=(15,6))
plt.scatter(reco_energy[0][1], signi[0][1], label=tag1+", odd id")
plt.scatter(reco_energy[1][1], signi[1][1], label=tag2+", odd id")
plt.scatter(reco_energy[0][1], signi[0][0], label=tag1+", even id")
plt.scatter(reco_energy[1][1], signi[1][0], label=tag2+", even id")

plt.xscale('log')

plt.ylim(2, 8)
plt.xlim(0.01, 100)
plt.legend()
plt.xlabel('Reconstructed energy / TeV')
plt.grid()
plt.show()  

In [ ]:
# Just a test plot to see how the minimum flux is found on even-numbered events, and applied to odd-numbered events
energyid = 2 # for energy bin
analysis_type = 0 # 0 source-independent, 1 source-dependent

print(10**(0.5*(logenergy_bins[energyid]+logenergy_bins[energyid+1])), "TeV")

if analysis_type == 0:
    start_bin = np.where(theta2bins>min_angle_cut[0][energyid])[0][0] - 1
else:
    start_bin = np.where(alphabins>min_angle_cut[1][energyid])[0][0] - 1

index = np.nanargmin(detectable_flux[analysis_type][0][energyid, :, start_bin:])
indices = list(np.unravel_index(index, detectable_flux[analysis_type][0][energyid, :, start_bin:].shape))
indices[1] += start_bin
# convert to indices in gammaness axis & angle axis

print("Minimum:", np.nanmin(detectable_flux[analysis_type][0][energyid, :, start_bin:]))
print(detectable_flux[analysis_type][0][energyid, indices[0], indices[1]])
print(detectable_flux[analysis_type][0][energyid, 
                                        indices[0]-3:indices[0]+4, 
                                        indices[1]-3:indices[1]+4])

gammaness_cut = gammaness_bins[indices[0]]
angular_cut = theta2bins[indices[1]+1]  # +1 because we want the bin's upper edge
if analysis_type == 1:
    angular_cut = alphabins[indices[1]+1]

print()
print("With the sample of even-numbered events:")
print('Minimum flux, gammaness & angular cut bins:', indices)
print('Cut values and minimum detectable flux (in fraction of Crab):',
      gammaness_cut, angular_cut, detectable_flux[analysis_type][0][energyid, indices[0], indices[1]])
print('Excess, Off events (for input sample), Li & Ma significance (in target t_obs for detectable flux):')
print('    ', cum_excess_events[analysis_type][0][energyid, indices[0], indices[1]],
      cum_off_events[analysis_type][0][energyid, indices[0], indices[1]],
      f'{lima_signi[analysis_type][0][energyid, indices[0], indices[1]]:.3f}')

plt.figure(figsize=(16,4))
if analysis_type == 0:
    plt.pcolormesh(theta2bins, gammaness_bins, detectable_flux[analysis_type][0][energyid], norm=colors.LogNorm())
else:
    plt.pcolormesh(alphabins, gammaness_bins, detectable_flux[analysis_type][0][energyid], norm=colors.LogNorm())

plt.colorbar()
plt.scatter([angular_cut], [gammaness_cut], marker='o', facecolors='none', edgecolors='red')

plt.ylabel('gammaness cut')
plt.xlabel('angular cut')

# plt.xlim(0, 0.5)
# plt.ylim(0.65, 0.75)
plt.show()

plt.figure(figsize=(16,4))
if analysis_type == 0:
    plt.pcolormesh(theta2bins, gammaness_bins, detectable_flux[analysis_type][1][energyid], norm=colors.LogNorm())
else:
    plt.pcolormesh(alphabins, gammaness_bins, detectable_flux[analysis_type][1][energyid], norm=colors.LogNorm())
plt.colorbar()
plt.scatter([angular_cut], [gammaness_cut], marker='o', facecolors='none', edgecolors='red')

plt.ylabel('gammaness cut')
plt.xlabel('angular cut')
# plt.xlim(0, 0.5)
# plt.ylim(0.65, 0.75)
plt.show()


print('With the sample of odd-numbered events:')
print('Applied cuts (from the other sample) and minimum detectable flux (in fraction of Crab):',
      gammaness_cut, angular_cut, detectable_flux[analysis_type][1][energyid, indices[0], indices[1]])
print('Excess, Off events (for input sample), Li & Ma significance (in target t_obs for detectable flux):')
print('    ', cum_excess_events[analysis_type][1][energyid, indices[0], indices[1]],
      cum_off_events[analysis_type][1][energyid, indices[0], indices[1]],
      f'{lima_signi[analysis_type][1][energyid, indices[0], indices[1]]:.3f}')

In [ ]:
#
# Function to find the cut indices (i.e. bin indices of the histos) which correspond to certain cuts
#
def find_bin_indices(gcut, angcut, analysis_type):
    # Find bin edge which is closest to the cut value:
    if analysis_type == 0:
        angcut_index = np.nanargmin(np.abs(theta2bins-angcut)) - 1     
    else:
        angcut_index = np.nanargmin(np.abs(alphabins-angcut)) - 1 

    gcut_index = np.nanargmin(np.abs(gammaness_bins-gcut))

    return gcut_index, angcut_index 

In [ ]:
#
# Plot a rebinned histogram (for better visualization)
#
def plot_rebinned(x, y, yerr, rebin, label):
    xx = np.array([0.5*(x[i]+x[i+rebin]) for i in range(0, len(x)-1, rebin)])
    yy = np.array([np.sum(y[i:i+rebin]) for i in range(0, len(y)-1, rebin)])
    yyerr = np.array([(np.sum(yerr[i:i+rebin]**2))**0.5 for i in range(0, len(yerr)-1, rebin)])
    plt.errorbar(xx, yy, yerr=yyerr, fmt='o', markersize=3, label=label)
    return xx, yy

In [ ]:
#
# Now we recalculate the sensitivities, and also check individual theta2 / Alphaplots:
#

target_obs_time = obs_time # = the same for which the cut optimization was done

# target_obs_time = 0.5 * u.h # CHANGE ONLY IN CASE YOU WANT TO CALCULATE SENSITIVITY 
                              # FOR DIFFERENT T_OBS, BUT *WITHOUT* RE-OPTIMIZING CUTS!!



norm_bins = np.array([[np.where(theta2bins>norm_range[0][0])[0][0],
                       np.where(theta2bins>norm_range[0][1])[0][0]],
                     [np.where(alphabins>norm_range[1][0])[0][0],
                      np.where(alphabins>norm_range[1][1])[0][0]]
                     ])



rebin_factor = np.array((len(logenergy_bins)-1)*[20])  # join bins in groups of rebin_factor, to make plots less noisy.
#rebin_factor = np.array((len(logenergy_bins)-1)*[60])  # join bins in groups of rebin_factor, to make plots less noisy.

rebin_factor[:3] = 120
rebin_factor[3:5] = 60
rebin_factor[14:] = 60

sensitivity = np.empty((2, len(logenergy_bins)-1))
sensitivity[:] = np.nan

delta_sensitivity = np.empty((2, 2, len(logenergy_bins)-1)) # separate upper and lower error bars
delta_sensitivity[:] = np.nan

# For 5-sigma condition only: 
# (NOTE! This is with the cuts optimized using all 3 conditions! In order to obtain the best sensitivity
# for just the 5 sigma condition, one has to set min_n_gammas=0 and min_backg_percent=0 BEFORE the cut 
# optimization!)
sensitivity_5s = np.zeros_like(sensitivity)
delta_sensitivity_5s = np.zeros_like(delta_sensitivity)
sensitivity_5s[:] = np.nan
delta_sensitivity_5s[:] = np.nan

reco_energy = np.zeros_like(sensitivity)
num_excess_events = np.zeros_like(sensitivity)
num_off_events = np.zeros_like(sensitivity)
reco_energy[:] = np.nan
num_excess_events[:] = np.nan
num_off_events[:] = np.nan

angular_cut = np.empty((2, 2, len(logenergy_bins)-1))
gammaness_cut = np.empty((2, 2, len(logenergy_bins)-1)) # analysis_type, odd_or_even, energy
angular_cut[:] = np.nan
gammaness_cut[:] = np.nan


for iebin in range(len(logenergy_bins)-1):

    recoE = 10**(0.5*(logenergy_bins[iebin]+logenergy_bins[iebin+1]))
    reco_energy[0][iebin] = recoE
    reco_energy[1][iebin] = recoE

    
    if (recoE < 0.016) | (recoE > 16):
        continue
    
    print(f'Energy: {recoE:.4f} Tev')    
    
    fig = plt.figure(figsize=(16, 5))
    
    for analysis_type in range(2):
    
        indices0 = cut_indices[analysis_type][0][iebin]
        indices1 = cut_indices[analysis_type][1][iebin]
        # indices0 and indices1 have 2 elements each: [0] is the gammaness cut, [1] is the angular cut

        if (indices0>0).all() and (indices1>0).all():
            # Valid gammanness & angular cuts (cut indices ==0 means no valid cuts could be determined!)
            nevts_on = (np.sum(on_events[analysis_type][0, iebin, indices0[0]:], axis=0) + 
                        np.sum(on_events[analysis_type][1, iebin, indices1[0]:], axis=0))

            nevts_off = (np.sum(off_events[analysis_type][0, iebin, indices0[0]:], axis=0) + 
                         np.sum(off_events[analysis_type][1, iebin, indices1[0]:], axis=0))
        else:
            nevts_on = None
            nevts_off = None
            
        if analysis_type == 0:
            fig.add_subplot(1, 2, 1+analysis_type)

            if nevts_on is None:
                print('No valid cuts found for source-independent analysis!')
                continue
            else:
                print(f'Gammaness cuts: {gammaness_bins[indices0[0]+1]:.4f}, {gammaness_bins[indices1[0]+1]:.4f}')
                print(f'Theta2 cuts: {theta2bins[indices0[1]+1]:.4f}, {theta2bins[indices1[1]+1]:.4f}')
                
            xx, yy = plot_rebinned(theta2bins, nevts_on, nevts_on**0.5, rebin_factor[iebin], '')
            xxoff, yyoff = plot_rebinned(theta2bins, nevts_off, nevts_off**0.5, rebin_factor[iebin], '')

            plt.plot([theta2bins[indices0[1]+1], theta2bins[indices0[1]+1]],  
                     [0, yy[int((indices0[1]+1)/rebin_factor[iebin])]], '--', color='tab:green')
            plt.plot([theta2bins[indices1[1]+1], theta2bins[indices1[1]+1]],  
                     [0, yy[int((indices1[1]+1)/rebin_factor[iebin])]], '--', color='tab:green')

            plt.xlim(0, 0.2)
            plt.ylim(yyoff.min()*0.9, yy.max()*1.1)
            plt.xlabel('Theta2 (deg2)')
            plt.ylabel('Events')

            angular_cut[analysis_type][0][iebin] = theta2bins[indices0[1]+1]
            angular_cut[analysis_type][1][iebin] = theta2bins[indices1[1]+1]

        else:
            fig.add_subplot(1, 2, 1+analysis_type)
            
            if nevts_on is None:
                print('No valid cuts found for source-dependent analysis!')
                continue
            else:
                print(f'Alpha cuts: {alphabins[indices0[1]+1]:.2f}, {alphabins[indices1[1]+1]:.2f}')
            
            xx, yy = plot_rebinned(alphabins, nevts_on, nevts_on**0.5, rebin_factor[iebin], '')
            xxoff, yyoff = plot_rebinned(alphabins, nevts_off, nevts_off**0.5, rebin_factor[iebin], '')

            plt.plot([alphabins[indices0[1]+1], alphabins[indices0[1]+1]],  
                     [0, yy[int((indices0[1]+1)/rebin_factor[iebin])]], '--', color='tab:green')
            plt.plot([alphabins[indices1[1]+1], alphabins[indices1[1]+1]],  
                     [0, yy[int((indices1[1]+1)/rebin_factor[iebin])]], '--', color='tab:green')
            plt.xlim(0, 60)
            plt.ylim(yyoff.min()*0.9, yy.max()*1.1)
            plt.xlabel('Alpha (deg)')
            plt.ylabel('Events')

            angular_cut[analysis_type][0][iebin] = alphabins[indices0[1]+1]
            angular_cut[analysis_type][1][iebin] = alphabins[indices1[1]+1]

        # Add up the backg numbers (odd and even events) in the normalization region, and the excess:
        off_in_norm_region = (cum_off_events[analysis_type][0, iebin, indices0[0], norm_bins[analysis_type][1]] +
                              cum_off_events[analysis_type][1, iebin, indices1[0], norm_bins[analysis_type][1]] -
                              cum_off_events[analysis_type][0, iebin, indices0[0], norm_bins[analysis_type][0]] -
                              cum_off_events[analysis_type][1, iebin, indices1[0], norm_bins[analysis_type][0]]
                              )
        excess_in_norm_region = (cum_excess_events[analysis_type][0, iebin, indices0[0], norm_bins[analysis_type][1]] +
                                 cum_excess_events[analysis_type][1, iebin, indices1[0], norm_bins[analysis_type][1]] -
                                 cum_excess_events[analysis_type][0, iebin, indices0[0], norm_bins[analysis_type][0]] -
                                 cum_excess_events[analysis_type][1, iebin, indices1[0], norm_bins[analysis_type][0]]
                                )
        off_norm_factor = 1
        if backg_normalization:
            off_norm_factor = (off_in_norm_region + excess_in_norm_region) / off_in_norm_region
            print('Off normalization for analysis type', analysis_type, ':', off_norm_factor)

            norm_min = (((off_in_norm_region + excess_in_norm_region) - 
                         (off_in_norm_region + excess_in_norm_region)**0.5) / 
                        (off_in_norm_region + off_in_norm_region**0.5))
            norm_max = (((off_in_norm_region + excess_in_norm_region) + 
                         (off_in_norm_region + excess_in_norm_region)**0.5) / 
                        (off_in_norm_region - off_in_norm_region**0.5))
            print(f'     {norm_min:.4f} to {norm_max:.4f}')


        gammaness_cut[analysis_type][0][iebin] = gammaness_bins[indices0[0]]
        gammaness_cut[analysis_type][1][iebin] = gammaness_bins[indices1[0]]

        # Add up the excess (and the off) for odd and even event_id's
        nexcess = (cum_excess_events[analysis_type][0, iebin, indices0[0], indices0[1]] +
                   cum_excess_events[analysis_type][1, iebin, indices1[0], indices1[1]])
        noff = (cum_off_events[analysis_type][0, iebin, indices0[0], indices0[1]] +
                cum_off_events[analysis_type][1, iebin, indices1[0], indices1[1]])


        nexcess = nexcess + noff * (1 - off_norm_factor)
        noff = noff * off_norm_factor


        flux = calc_flux_3conditions(np.array([nexcess]), np.array([noff]), 
                                     min_signi, min_exc, min_off_events, alpha,
                                     target_obs_time, livetimes[analysis_type],
                                     min_n_gammas, min_backg_percent)

        if analysis_type == 0:
            print(f'Results (source-indep): Nexc={nexcess}, Noff={noff},  Flux/CU={flux[0]:.4f}')
        else:
            print(f'Results (source-dep): Nexc={nexcess}, Noff={noff},  Flux/CU={flux[0]:.4f}')

        sensitivity[analysis_type][iebin] = flux[0]

        # Assume background systematics and statistical fluctuation of excess go in same direction:
        max_excess = nexcess + backg_syst * noff + (nexcess + 2*noff)**0.5
        min_excess = nexcess - backg_syst * noff - (nexcess + 2*noff)**0.5

        flux_minus = calc_flux_3conditions(np.array([max_excess]), 
                                           np.array([noff]), 
                                           0, 0, 0, alpha,
                                           target_obs_time, livetimes[analysis_type],
                                           min_n_gammas, min_backg_percent)
        flux_plus = calc_flux_3conditions(np.array([min_excess]), 
                                          np.array([noff]), 
                                          0, 0, 0, alpha,
                                          target_obs_time, livetimes[analysis_type],
                                          min_n_gammas, min_backg_percent)

        delta_sensitivity[analysis_type][1][iebin] = flux_plus[0] - flux[0]
        delta_sensitivity[analysis_type][0][iebin] = flux[0] - flux_minus[0]


        # Now only with the 5-sigma condition (remove req for 10 excess events & 5% of backg):

        flux_5s = calc_flux_3conditions(np.array([nexcess]), np.array([noff]), 
                                        min_signi, min_exc, min_off_events, alpha,
                                        target_obs_time, livetimes[analysis_type],
                                        0, 0)
        sensitivity_5s[analysis_type][iebin] = flux_5s[0]
        flux_5s_minus = calc_flux_3conditions(np.array([max_excess]), 
                                              np.array([noff]), 
                                              0, 0, 0, alpha,
                                              target_obs_time, livetimes[analysis_type],
                                              0, 0)
        flux_5s_plus = calc_flux_3conditions(np.array([min_excess]), 
                                             np.array([noff]), 
                                             0, 0, 0, alpha,
                                             target_obs_time, livetimes[analysis_type],
                                             0, 0)

        delta_sensitivity_5s[analysis_type][1][iebin] = flux_5s_plus[0] - flux_5s[0]
        delta_sensitivity_5s[analysis_type][0][iebin] = flux_5s[0] - flux_5s_minus[0]


        num_excess_events[analysis_type][iebin] = nexcess
        num_off_events[analysis_type][iebin] = noff

        # plt.ylim(0, 250)

        plt.grid()
        # plt.legend()

    plt.show()
    print()

In [ ]:
#
# FINAL SENSITIVITY PLOTS
#
plt.figure(figsize=(8,6))


plt.fill_between(reco_energy[0][:-1], 
                 (sensitivity[0]-delta_sensitivity[0][0])[:-1], 
                 (sensitivity[0]+delta_sensitivity[0][1])[:-1], alpha=0.4, color='tab:blue')


plt.fill_between(reco_energy[1][:-1], 
                 (sensitivity[1]-delta_sensitivity[1][0])[:-1], 
                 (sensitivity[1]+delta_sensitivity[1][1])[:-1], alpha=0.4, color='tab:orange')


plt.errorbar(reco_energy[0], sensitivity[0],
             yerr=delta_sensitivity[0], marker='o', color='tab:blue', ls='none', markersize=4,
             label='LST-1 (source-independent)')

plt.errorbar(reco_energy[1], sensitivity[1],
             yerr=delta_sensitivity[1], marker='o', color='tab:orange', ls='none', markersize=4,
             label='LST-1 (source-dependent)')


plot_MAGIC_sensitivity_fraction()

plt.xscale('log')
plt.yscale('log')
plt.xlabel('E$_{reco}$ / TeV')
plt.ylabel('Fraction of Crab nebula flux')
plt.ylim(0.01, 50)
plt.xlim(0.02, 12)
plt.legend()
plt.show()

In [ ]:
# Write out the results:
Output_filename = "Sensitivity_output.csv"
np.savetxt(Output_filename, [reco_energy[0], 
                             sensitivity[0], 
                             delta_sensitivity[0][0],
                             delta_sensitivity[0][1],
                             sensitivity[1], 
                             delta_sensitivity[1][0],
                             delta_sensitivity[1][1],
                             sensitivity_5s[0], 
                             delta_sensitivity_5s[0][0],
                             delta_sensitivity_5s[0][1],
                             sensitivity_5s[1], 
                             delta_sensitivity_5s[1][0],
                             delta_sensitivity_5s[1][1]],
           fmt='%.5e', delimiter=',',
           header = 'E(TeV)  SI_sensitivity, SI_delta_sensitivity_low, SI_delta_sensitivity_high, '+
           'SD_sensitivity, SD_delta_sensitivity_low, SD_delta_sensitivity_high, '+
           'SI_sensitivity_5s, SI_delta_sensitivity_5s_low, SI_delta_sensitivity_5s_high, '+
           'SD_sensitivity_5s, SD_delta_sensitivity_5s_low, SD_delta_sensitivity_5s_high')

In [ ]:
# REDO PLOT FROM CSV FILE:

plt.figure(figsize=(8,6))

data = np.loadtxt(Output_filename, delimiter=',')
plt.fill_between(data[0][:-1], 
                 (data[1]-data[2])[:-1], 
                 (data[1]+data[3])[:-1], alpha=0.4, color='tab:blue')
plt.fill_between(data[0][:-1], 
                 (data[4]-data[5])[:-1], 
                 (data[4]+data[6])[:-1], alpha=0.4, color='tab:orange')

plt.errorbar(data[0], data[1],
             yerr=[data[2], data[3]], marker='o', color='tab:blue', ls='none', markersize=4,
             label='LST-1 (source-independent)')
plt.errorbar(data[0], data[4],
             yerr=[data[5], data[6]], marker='o', color='tab:orange', ls='none', markersize=4,
             label='LST-1 (source-dependent)')

plot_MAGIC_sensitivity_fraction()

plt.xscale('log')
plt.yscale('log')
plt.xlabel('E$_{reco}$ / TeV')
plt.ylabel('Fraction of Crab nebula flux')
plt.xlim(0.02, 12)
plt.ylim(0.01, 50)
plt.legend()
plt.show()

In [ ]:
np.nancumsum(num_excess_events[0][::-1])[::-1]

In [ ]:
# Integral numbers of events
fig = plt.figure(figsize=(16, 6))

fig.add_subplot(1, 2, 1)
plt.plot(10**logenergy_bins[:-1], np.nancumsum(num_excess_events[0][::-1])[::-1])
plt.plot(10**logenergy_bins[:-1], np.nancumsum(num_excess_events[1][::-1])[::-1])
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Ereco / TeV')
plt.ylabel('Number of excess events, integrated above Ereco')
plt.xlim(0.01, 30)

fig.add_subplot(1, 2, 2)
plt.plot(10**logenergy_bins[:-1], np.nancumsum(num_off_events[0][::-1])[::-1])
plt.plot(10**logenergy_bins[:-1], np.nancumsum(num_off_events[1][::-1])[::-1])
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Ereco / TeV')
plt.ylabel('Number of OFF events, integrated above Ereco')
plt.xlim(0.01, 30)
plt.show()

In [ ]:
#
# Calculate integral sensitivity, using the same optimized cuts in each bin of Ereco.
# NOTE: at low E differential sensitivities may be similar, yet the integral ones differ, because of
# the different rates of excess and background (resulting in different conditions determining the
# flux sensitivity value). Cuts are not re-optimized!
# NOTE anyway that only the best integral sensitivity (in terms of Crab fraction) is relevant!
#

integral_sensitivity = np.zeros_like(sensitivity)
integral_sensitivity[:] = np.nan

for analysis_type in range(2):
    
    for iebin in range(len(logenergy_bins)-1):
        total_excess = np.nansum(num_excess_events[analysis_type][iebin:])
        total_off = np.nansum(num_off_events[analysis_type][iebin:])
    
        flux = calc_flux_3conditions(np.array([total_excess]), np.array([total_off]), 
                                     min_signi, min_exc, min_off_events, alpha,
                                     target_obs_time, livetimes[analysis_type],
                                     min_n_gammas, min_backg_percent)

        integral_sensitivity[analysis_type][iebin] = flux[0]    

In [ ]:
plt.figure(figsize=(12,6))

plt.plot(10**logenergy_bins[:-1], integral_sensitivity[0], label='Source-independent')
plt.plot(10**logenergy_bins[:-1], integral_sensitivity[1], label='Source-dependent')

plt.xscale('log')
plt.yscale('log')
#plt.ylim(0.01, 0.5)
plt.xlabel('E$_{reco}$ / TeV')
plt.ylabel('Integral sensitivity (fraction of Crab nebula flux)')
plt.grid()
plt.legend()
plt.show()

print(f'Best integral sensitivity, source-indep: {np.nanmin(integral_sensitivity[0]):.4f} C.U.')
print(f'Best integral sensitivity, source-dep: {np.nanmin(integral_sensitivity[1]):.4f} C.U.')

In [ ]:
np.set_printoptions(precision=4)
print("Energies (TeV):", np.array(reco_energy[0]))

print()
for analysis_type in range(2):
    if analysis_type == 0:
        print("Values for source-independent analysis:")
    else:
        print("Values for source-dependent analysis:")
    print("Sensitivity:", sensitivity[analysis_type])
    print()
    print("Nexcess:", num_excess_events[analysis_type])
    print()
    print("Noff:", num_off_events[analysis_type])
    
    print()
    print("Gammaness cut (even):", np.where(np.isnan(sensitivity[analysis_type]), np.nan, 
                                            gammaness_cut[analysis_type][0]))
    print()
    print("Gammaness cut (odd):", np.where(np.isnan(sensitivity[analysis_type]), np.nan,
                                           gammaness_cut[analysis_type][1]))
    print()
    print("Angular cut (even):", np.where(np.isnan(sensitivity[analysis_type]), np.nan,
                                          angular_cut[analysis_type][0]))
    print()
    print("Angular cut (odd):", np.where(np.isnan(sensitivity[analysis_type]), np.nan,
                                         angular_cut[analysis_type][1]))
        
    print('\n'*3)

In [ ]:
[angular_cut[0][1], angular_cut[1][1]]

In [ ]:
[gammaness_cut[0][1], gammaness_cut[1][1]]

In [ ]:
plt.plot(reco_energy[0], angular_cut[0][0], label='even')
plt.plot(reco_energy[1], angular_cut[0][1], label='odd')
plt.xscale('log')
plt.grid()
plt.ylabel('theta2 cut / deg2')
plt.xlabel('E / TeV')
plt.legend()
plt.show()

plt.plot(reco_energy[0], angular_cut[1][0], label='even')
plt.plot(reco_energy[1], angular_cut[1][1], label='odd')
plt.xscale('log')
plt.grid()
plt.ylabel('Alpha cut / deg')
plt.xlabel('E / TeV')
plt.legend()
plt.show()

In [ ]:
plt.plot(reco_energy[0], num_excess_events[0], label='source-independent')
plt.plot(reco_energy[1], num_excess_events[1], label='source-dependent')
plt.xscale('log')
plt.yscale('log')
plt.grid()
plt.ylabel('Excess events')
plt.xlabel('E / TeV')
plt.legend()
plt.show()

plt.plot(reco_energy[0], num_off_events[0], label='source-independent')
plt.plot(reco_energy[1], num_off_events[1], label='source-dependent')
plt.xscale('log')
plt.yscale('log')
plt.grid()
plt.ylabel('Off events')
plt.xlabel('E / TeV')
plt.legend()
plt.show()


In [ ]:
num_off_events[0]

In [ ]:
num_excess_events[0]

In [ ]:
plt.plot(reco_energy[0], gammaness_cut[0][0], label='even')
plt.plot(reco_energy[0], gammaness_cut[0][1], label='odd')
plt.xscale('log')
plt.grid()
plt.ylabel('Gammaness')
plt.xlabel('E / TeV')
plt.legend()
plt.show()

plt.plot(reco_energy[1], gammaness_cut[1][0], label='even')
plt.plot(reco_energy[1], gammaness_cut[1][1], label='odd')
plt.xscale('log')
plt.grid()
plt.ylabel('Gammaness')
plt.xlabel('E / TeV')
#plt.ylim(0.9, 1.05)
plt.legend()
plt.show()

In [ ]:
import scipy.integrate as integrate
def dfde(x):
    return CRAB_MAGIC_JHEAP2015(x*u.TeV).to_value(1/(u.TeV*u.s*u.cm**2))

In [ ]:
dfde(1.)

In [ ]:
etev = reco_energy[0] * u.TeV
plt.plot(etev, CRAB_MAGIC_JHEAP2015(etev))
plt.xscale('log')
plt.yscale('log')
plt.show()

logenergy_bins[iebin]+logenergy_bins[iebin+1]

rate_per_s_m2 = 1e4*np.array([integrate.quad(dfde, 10**a, 10**b)[0] 
                              for a, b in zip(logenergy_bins[:-1], logenergy_bins[1:])])
rate_per_s_m2

In [ ]:
plt.plot(reco_energy[0], num_excess_events[0]/livetimes[0]/rate_per_s_m2, label='source-independent')
plt.plot(reco_energy[1], num_excess_events[1]/livetimes[1]/rate_per_s_m2, label='source-dependent')
plt.xscale('log')
plt.yscale('log')
plt.grid()
plt.ylabel('Aeff(m2)')
plt.xlabel('E / TeV')
plt.legend()
plt.show()